In [48]:
%%writefile Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM python:3.9

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip install numpy scipy scikit-learn pandas flask gevent gunicorn catboost sagemaker-training sagemaker-inference && \
        rm -rf /root/.cache

# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# keeps Python from writing the .pyc files which are unnecessary in this case. We also update
# PATH so that the train and serve programs are found when the container is invoked.

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the program in the image
COPY catboost /opt/program
RUN chmod -R 777 /opt/program
WORKDIR /opt/program


Overwriting Dockerfile


In [49]:
import boto3
import sagemaker

from sagemaker import get_execution_role

role = get_execution_role()

ecr_namespace = 'custom-image/'
prefix = 'catboost-image'

ecr_repository_name = ecr_namespace + prefix
account_id = role.split(':')[4]
region = boto3.Session().region_name
sess = sagemaker.session.Session()
bucket = sess.default_bucket()

print('Account: {}'.format(account_id))
print('Region: {}'.format(region))
print('Role: {}'.format(role))
print('S3 Bucket: {}'.format(bucket))

Account: 405392309789
Region: ap-southeast-1
Role: arn:aws:iam::405392309789:role/sagemaker-risk-role
S3 Bucket: sagemaker-ap-southeast-1-405392309789


In [50]:
%%writefile build_and_push.sh

ACCOUNT_ID=$1
REGION=$2
REPO_NAME=$3

if [[ $REGION =~ ^cn.* ]]
then
    FULLNAME="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com.cn/${REPO_NAME}:latest"
else
    FULLNAME="${ACCOUNT_ID}.dkr.ecr.${REGION}.amazonaws.com/${REPO_NAME}:latest"
fi

echo $FULLNAME

docker build -f Dockerfile -t $REPO_NAME .

docker tag $REPO_NAME $FULLNAME

$(aws ecr get-login --no-include-email --registry-ids $ACCOUNT_ID)

aws ecr describe-repositories --repository-names $REPO_NAME || aws ecr create-repository --repository-name $REPO_NAME

docker push $FULLNAME

Overwriting build_and_push.sh


In [51]:
! bash build_and_push.sh $account_id $region $ecr_repository_name

405392309789.dkr.ecr.ap-southeast-1.amazonaws.com/custom-image/catboost-image:latest
Sending build context to Docker daemon  69.12kB
Step 1/9 : FROM python:3.9
 ---> 618db5c0a924
Step 2/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 1126a7928835
Step 3/9 : RUN pip install numpy scipy scikit-learn pandas flask gevent gunicorn catboost sagemaker-training sagemaker-inference &&         rm -rf /root/.cache
 ---> Using cache
 ---> 45519208a2e9
Step 4/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 67b557c757e1
Step 5/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> e941deb1aafa
Step 6/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 6e110dd13640
Step 7/9 : COPY catboost /opt/program
 ---> Using cache
 ---> f8df00186273
Step 8/9 : RUN chmod -R 777 /opt/program
 ---> Using cache
 ---> 0c54d96e17d8
Step 9/9 : WORKDIR /opt/program